# **AutoMarket Agent**

# An autonomous AI agent using LangGraph to perform deep market research, financial analysis, and investment reporting without human intervention.

This cell installs the necessary Python dependencies for the project, including `langgraph`, `langchain`, `langchain_groq`, `langchain_community`, `tavily-python`, `yfinance`, `beautifulsoup4`, and `termcolor`.

In [1]:
!pip install -q langgraph langchain langchain_groq langchain_community tavily-python yfinance beautifulsoup4 termcolor
print("Dependencies installed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
Dependencies installed!


This cell imports the `os` module and `userdata` from `google.colab` to securely retrieve and set API keys as environment variables for Groq and Tavily.

In [2]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

This cell defines the `AgentState` class, a `TypedDict` that outlines the structure of the data passed between different nodes in the LangGraph workflow. It includes fields for company, stock ticker, news data, financial data, competitor data, analysis, and the final report.

In [3]:
from typing import TypedDict, List

class AgentState(TypedDict):
    company: str
    stock_ticker: str
    news_data: List[str]
    financial_data: dict
    competitor_data: str
    analysis: str
    final_report: str

This cell initializes the tools used by the agents. It sets up `TavilySearchResults` for web searching and defines a custom tool `get_stock_data` using `yfinance` to fetch stock information (price, market cap, PE ratio, revenue growth) for a given ticker.

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
import yfinance as yf

# 1. Search Tool
search_tool = TavilySearchResults(max_results=3)

# 2. Finance Tool (Stock Data)
@tool
def get_stock_data(ticker: str):
    """
    Get stock data for a given ticker symbol (e.g., TSLA, AAPL).
    Returns price, market cap, and PE ratio.
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        return {
            "current_price": info.get('currentPrice'),
            "market_cap": info.get('marketCap'),
            "pe_ratio": info.get('trailingPE'),
            "revenue_growth": info.get('revenueGrowth')
        }
    except Exception as e:
        return {"error": str(e)}

# List of tools to bind to the LLM
tools = [search_tool, get_stock_data]

/tmp/ipython-input-4255193903.py:6: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(max_results=3)


This cell configures the Large Language Model (LLM) using `ChatGroq` with the `llama-3.3-70b-versatile` model. It then binds the previously defined tools (`search_tool` and `get_stock_data`) to the LLM, making them available for the model to use.

In [9]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)
llm_with_tools = llm.bind_tools(tools)

**We will create three distinct "workers":**

1. Market Researcher: Finds news and competitors.
2. Financial Analyst: Gets stock prices and analyzes numbers.
3. Report Writer: Compiles everything into the final markdown report.




In [11]:
from langchain_core.messages import HumanMessage, SystemMessage

# Node 1: Market Researcher
def market_research_node(state: AgentState):
    print(f"🕵️ RESEARCHER: Looking up news and web for {state['company']}...")

    # Simple search invocation
    results = search_tool.invoke(f"{state['company']} recent news and competitors")

    # Store results in state
    news_summary = [res['content'] for res in results]
    return {"news_data": news_summary}

# Node 2: Financial Analyst
def financial_analyst_node(state: AgentState):
    print(f"📊 ANALYST: Analyzing stock data for {state['company']}...")

    # Let the LLM decide the ticker based on company name
    # (In a real app, we'd use a tool for this, but let's hardcode or infer for simplicity)
    # For this demo, we assume the user provides the Ticker or we ask LLM to guess it.
    # Let's ask the LLM to extract the ticker first.

    prompt = f"What is the stock ticker for {state['company']}? Respond ONLY with the ticker symbol (e.g., AAPL)."
    ticker = llm.invoke(prompt).content.strip()

    print(f"   -> Identified Ticker: {ticker}")

    # Call the tool manually for precision
    financials = get_stock_data.invoke(ticker)

    analysis_prompt = f"""
    Analyze these financials for {state['company']}: {financials}.
    Is this a healthy company? Keep it brief.
    """
    analysis = llm.invoke(analysis_prompt).content

    return {"stock_ticker": ticker, "financial_data": financials, "analysis": analysis}

# Node 3: Report Writer
def report_writer_node(state: AgentState):
    print(f"✍️ WRITER: Compiling final report...")

    prompt = f"""
    You are a Wall Street Investment Banker. Write a 1-page investment report for {state['company']}.

    Use the following data:
    1. News: {state['news_data']}
    2. Financials: {state['financial_data']}
    3. Analyst Opinion: {state['analysis']}

    Format nicely in Markdown with headers. Include a 'Buy/Sell/Hold' recommendation.
    """

    report = llm.invoke(prompt).content
    return {"final_report": report}

This cell sets up the LangGraph workflow. It initializes a `StateGraph` with the `AgentState`, adds the three defined nodes (`researcher`, `analyst`, `writer`), defines the sequential edges connecting them (researcher -> analyst -> writer -> END), and finally compiles the graph into an executable application.

In [12]:
from langgraph.graph import StateGraph, END

# Initialize Graph
workflow = StateGraph(AgentState)

# Add Nodes
workflow.add_node("researcher", market_research_node)
workflow.add_node("analyst", financial_analyst_node)
workflow.add_node("writer", report_writer_node)

# Define Edges (The Flow)
# Start -> Researcher -> Analyst -> Writer -> End
workflow.set_entry_point("researcher")
workflow.add_edge("researcher", "analyst")
workflow.add_edge("analyst", "writer")
workflow.add_edge("writer", END)

# Compile
app = workflow.compile()

This cell initiates and runs the compiled LangGraph application. It defines the target company, initializes the graph with this company, invokes the graph to execute the workflow, and then prints the final generated investment report in Markdown format.

In [18]:
from IPython.display import Markdown

# Input
company_name = "tesla"

print(f"🚀 Starting AutoMarket-Agent for: {company_name}\n")

# Run the graph
inputs = {"company": company_name}
result = app.invoke(inputs)

# Output the result
print("\n\n✅ PROCESS COMPLETE. REPORT GENERATED:\n")
display(Markdown(result["final_report"]))

🚀 Starting AutoMarket-Agent for: tesla

🕵️ RESEARCHER: Looking up news and web for tesla...
📊 ANALYST: Analyzing stock data for tesla...
   -> Identified Ticker: TSLA
✍️ WRITER: Compiling final report...


✅ PROCESS COMPLETE. REPORT GENERATED:



### Investment Report: Tesla
#### Overview
Tesla, Inc. is a leading electric vehicle (EV) manufacturer that has recently faced increased competition, particularly from Chinese companies such as BYD. Despite being a pioneer in the EV market, Tesla has lost its position as the world's leading EV manufacturer and is experiencing a sales slump.

#### Market Analysis
The EV market is becoming increasingly competitive, with many established brands such as Honda, Nissan, and GM entering the space. However, some of these companies have scaled back their EV ambitions, while others, like Hyundai, are taking a mixed approach. The loss of government incentives has also impacted the market, with sales of electrified vehicles projected to decline.

#### Financials
The company's current financials are as follows:
* Current Price: $485.40
* Market Capitalization: $1.61 trillion
* P/E Ratio: 332.46
* Revenue Growth: 11.6%

#### Analyst Opinion
Based on the provided financials, Tesla's high P/E ratio indicates a potentially overvalued stock. However, the company's moderate revenue growth is a positive sign, and its large market capitalization suggests a stable and established company.

#### Recommendation
**Hold**
While Tesla's financials indicate a healthy company with growing revenue, the high P/E ratio and increased competition in the EV market are causes for caution. Additionally, the sales slump and loss of government incentives may impact the company's short-term performance. Therefore, we recommend a hold position on Tesla stock, as the company navigates the evolving EV market and works to regain its competitive edge.

#### Risks and Opportunities
* Risks:
	+ Increased competition from established brands and new entrants
	+ Loss of government incentives impacting sales
	+ High P/E ratio indicating potential overvaluation
* Opportunities:
	+ Growing demand for EVs and sustainable energy solutions
	+ Tesla's established brand and loyal customer base
	+ Potential for innovation and expansion into new markets, such as freight trucks and driverless technology